In [1]:
# import os
# import pandas as pd
# from shutil import copy2

# def convert_to_yolo(row, width, height):
#     x1 = row['Roi.X1']
#     y1 = row['Roi.Y1']
#     x2 = row['Roi.X2']
#     y2 = row['Roi.Y2']

#     box_width = x2 - x1
#     box_height = y2 - y1
#     center_x = x1 + box_width / 2
#     center_y = y1 + box_height / 2

#     center_x /= width
#     center_y /= height
#     box_width /= width
#     box_height /= height

#     return center_x, center_y, box_width, box_height

# def process_csv(csv_path, images_root_dir, output_images_dir, output_labels_dir):
#     data_csv = pd.read_csv(csv_path)
#     for index, row in data_csv.iterrows():
#         image_path = row['Path']
#         image_name = os.path.basename(image_path)
#         txt_name = image_name.replace('.png', '.txt')

#         width, height = row['Width'], row['Height']
#         center_x, center_y, box_width, box_height = convert_to_yolo(row, width, height)
#         cls = row['ClassId']

#         src = os.path.join(images_root_dir, image_path)
#         dst = os.path.join(output_images_dir, image_name)

#         os.makedirs(output_images_dir, exist_ok=True)

#         if os.path.exists(src):
#             copy2(src, dst)
#         else:
#             continue

#         label_path = os.path.join(output_labels_dir, txt_name)
#         os.makedirs(output_labels_dir, exist_ok=True)
#         with open(label_path, 'w') as f:
#             f.write(f'{cls} {center_x} {center_y} {box_width} {box_height}\n')

# train_csv_path = '../input/gtsrb-german-traffic-sign/Train.csv'
# test_csv_path = '../input/gtsrb-german-traffic-sign/Test.csv'
# images_root_dir = '../input/gtsrb-german-traffic-sign'  
# output_train_images_dir = 'datasets/train/images'
# output_train_labels_dir = 'datasets/train/labels'
# output_test_images_dir = 'datasets/val/images'
# output_test_labels_dir = 'datasets/val/labels'

# process_csv(train_csv_path, images_root_dir, output_train_images_dir, output_train_labels_dir)
# process_csv(test_csv_path, images_root_dir, output_test_images_dir, output_test_labels_dir)


In [2]:
# !pip install ultralytics==8.0.196

# from IPython import display
# display.clear_output()

# import ultralytics
# ultralytics.checks()

In [3]:
# !nvidia-smi

In [4]:
# !yolo detect mode=train data=/kaggle/input/data-yaml/data.yaml model=yolov8s.yaml epochs=20 imgsz=512 batch=64

In [5]:
# !yolo detect train data=/kaggle/input/data-yaml/data.yaml model=yolov8n.pt epochs=20 imgsz=800 batch=64


In [6]:
# import os
# from ultralytics import YOLO

# data_path = '/kaggle/input/data-yaml/data.yaml'
# model_path = 'yolov8s.pt'

# model = YOLO(model_path)

# model.train(
#     data=data_path,
#     epochs=40,
#     imgsz=640,
#     batch=64,
#     fliplr=0.0,    # Hindari flipping horizontal (mirroring)
#     flipud=0.0,    # Hindari flipping vertikal (mirroring)
#     degrees=0.0,   # Hindari rotasi
# )


# Bagian II

## Latih Model dengan Dataset yg Beragam (Dataset dengan Jarak Jauh dan lebih dari 1 Kelas pada 1 Gambar)

In [7]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.33 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5689.3/8062.4 GB disk)


In [8]:
# import os
# import shutil
# import cv2
# from ultralytics import YOLO

# # Muat model YOLOv8 yang sudah dilatih
# model = YOLO('/kaggle/input/trafficsign/other/yolo/1/best11.pt')

# # Fungsi untuk memperbesar bounding box
# def enlarge_bbox(bbox, scale=1.2):
#     center_x, center_y, width, height = bbox
#     new_width = width * scale
#     new_height = height * scale
#     return [center_x, center_y, new_width, new_height]

# # Fungsi untuk memprediksi dan memperbarui label
# def predict_and_update_label(image_path, label_path, model, output_image_path, output_label_path):
#     image = cv2.imread(image_path)
#     height, width = image.shape[:2]

#     # Baca label yang ada
#     with open(label_path, 'r') as file:
#         lines = file.readlines()

#     new_labels = []

#     for line in lines:
#         parts = line.strip().split()
#         class_id = int(parts[0])
#         center_x = float(parts[1]) * width
#         center_y = float(parts[2]) * height
#         bbox_width = float(parts[3]) * width
#         bbox_height = float(parts[4]) * height

#         # Perbesar bounding box
#         bbox = [center_x, center_y, bbox_width, bbox_height]
#         enlarged_bbox = enlarge_bbox(bbox)

#         # Crop image sesuai bounding box yang diperbesar
#         x_min = max(0, int(enlarged_bbox[0] - enlarged_bbox[2] / 2))
#         y_min = max(0, int(enlarged_bbox[1] - enlarged_bbox[3] / 2))
#         x_max = min(width, int(enlarged_bbox[0] + enlarged_bbox[2] / 2))
#         y_max = min(height, int(enlarged_bbox[1] + enlarged_bbox[3] / 2))

#         cropped_image = image[y_min:y_max, x_min:x_max]

#         # Prediksi menggunakan YOLOv8 pada gambar yang sudah di-crop
#         results = model(cropped_image)

#         for result in results:
#             for box in result.boxes:
#                 cls = box.cpu().cls.numpy()[0]
#                 conf = box.cpu().conf.numpy()[0]
#                 x_center, y_center, w, h = box.cpu().xywh.numpy()[0]

#                 # Transformasi kembali koordinat ke skala asli gambar
#                 x_center = (x_center + x_min) / width
#                 y_center = (y_center + y_min) / height
#                 w /= width
#                 h /= height

#                 if conf > 0.5:  # Confidence threshold
#                     new_labels.append(f'{int(cls)} {x_center} {y_center} {w} {h}')

#     # Simpan hasil prediksi baru
#     with open(output_label_path, 'w') as out_file:
#         for label in new_labels:
#             out_file.write(label + '\n')

#     # Simpan gambar asli ke direktori output
#     cv2.imwrite(output_image_path, image)

# # Path ke direktori dengan gambar dan label
# data_dir = '/kaggle/input/traffic-signs-dataset-in-yolo-format/ts/ts'
# output_dir = '/kaggle/working/dataset2'  # Ganti dengan path ke direktori output

# # Buat direktori output jika belum ada
# test_dir = os.path.join(output_dir, 'test')
# train_dir = os.path.join(output_dir, 'train')
# os.makedirs(test_dir, exist_ok=True)
# os.makedirs(train_dir, exist_ok=True)

# # Baca daftar file test dari test.txt
# with open('/kaggle/input/traffic-signs-dataset-in-yolo-format/test.txt', 'r') as file:
#     test_files = {os.path.basename(line.strip()) for line in file.readlines()}

# # Proses setiap file dalam direktori
# for filename in os.listdir(data_dir):
#     if filename.endswith(('.jpg', '.jpeg', '.png')):
#         image_path = os.path.join(data_dir, filename)
#         label_path = os.path.join(data_dir, os.path.splitext(filename)[0] + '.txt')
        
#         if filename in test_files:
#             output_image_path = os.path.join(test_dir, filename)
#             output_label_path = os.path.join(test_dir, os.path.splitext(filename)[0] + '.txt')
#         else:
#             output_image_path = os.path.join(train_dir, filename)
#             output_label_path = os.path.join(train_dir, os.path.splitext(filename)[0] + '.txt')
        
#         predict_and_update_label(image_path, label_path, model, output_image_path, output_label_path)


In [9]:
import os
import cv2
import numpy as np
from collections import defaultdict
from ultralytics import YOLO

# Muat model YOLOv8 yang sudah dilatih
model = YOLO('/kaggle/input/trafficsign/other/yolo/1/best11.pt')  # Ganti dengan path ke model YOLOv8 Anda

# Fungsi untuk memperbesar bounding box
def enlarge_bbox(bbox, scale=1.2):
    center_x, center_y, width, height = bbox
    new_width = width * scale
    new_height = height * scale
    return [center_x, center_y, new_width, new_height]

# Fungsi untuk memprediksi dan memperbarui label
def predict_and_update_label(image_path, label_path, model, output_image_path, output_label_path):
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # Baca label yang ada
    with open(label_path, 'r') as file:
        lines = file.readlines()

    new_labels = []
    class_counts = defaultdict(int)  # Dictionary untuk menghitung jumlah objek per kelas

    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        center_x = float(parts[1]) * width
        center_y = float(parts[2]) * height
        bbox_width = float(parts[3]) * width
        bbox_height = float(parts[4]) * height

        # Perbesar bounding box
        bbox = [center_x, center_y, bbox_width, bbox_height]
        enlarged_bbox = enlarge_bbox(bbox)

        # Crop image sesuai bounding box yang diperbesar
        x_min = max(0, int(enlarged_bbox[0] - enlarged_bbox[2] / 2))
        y_min = max(0, int(enlarged_bbox[1] - enlarged_bbox[3] / 2))
        x_max = min(width, int(enlarged_bbox[0] + enlarged_bbox[2] / 2))
        y_max = min(height, int(enlarged_bbox[1] + enlarged_bbox[3] / 2))

        cropped_image = image[y_min:y_max, x_min:x_max]

        # Prediksi menggunakan YOLOv8 pada gambar yang sudah di-crop
        results = model(cropped_image)

        for result in results:
            for box in result.boxes:
                cls = box.cpu().cls.numpy()[0]
                conf = box.cpu().conf.numpy()[0]
                x_center, y_center, w, h = box.cpu().xywh.numpy()[0]

                # Transformasi kembali koordinat ke skala asli gambar
                x_center = (x_center + x_min) / width
                y_center = (y_center + y_min) / height
                w /= width
                h /= height

                if conf > 0.5:  # Confidence threshold
                    new_labels.append(f'{int(cls)} {x_center} {y_center} {w} {h}')
                    class_counts[int(cls)] += 1

    # Simpan hasil prediksi baru
    with open(output_label_path, 'w') as out_file:
        for label in new_labels:
            out_file.write(label + '\n')

    # Simpan gambar asli ke direktori output
    cv2.imwrite(output_image_path, image)

    return class_counts

# Path ke direktori dengan gambar dan label
data_dir = '/kaggle/input/traffic-signs-dataset-in-yolo-format/ts/ts'
output_dir = '/kaggle/working/dataset2'  # Ganti dengan path ke direktori output

# Buat direktori output jika belum ada
os.makedirs(output_dir, exist_ok=True)

# Dictionary untuk mengumpulkan jumlah objek per kelas dari semua gambar
total_class_counts = defaultdict(int)

# Proses setiap file dalam direktori
for filename in os.listdir(data_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(data_dir, filename)
        label_path = os.path.join(data_dir, os.path.splitext(filename)[0] + '.txt')
        output_image_path = os.path.join(output_dir, filename)  # Gambar asli disimpan dengan nama yang sama
        output_label_path = os.path.join(output_dir, os.path.splitext(filename)[0] + '.txt')
        
        class_counts = predict_and_update_label(image_path, label_path, model, output_image_path, output_label_path)
        
        # Tambahkan jumlah objek per kelas dari gambar ini ke total_class_counts
        for cls_id, count in class_counts.items():
            total_class_counts[cls_id] += count

# Tampilkan jumlah objek per kelas
print("Total object counts per class:")
for cls_id, count in total_class_counts.items():
    print(f"Class {cls_id}: {count}")

from IPython import display
display.clear_output()

In [10]:
import os
import shutil
from collections import defaultdict
import random

# Path ke direktori dengan gambar dan label
data_dir = '/kaggle/working/dataset2'
output_dir = '/kaggle/working/dataset2'  # Ganti dengan path ke direktori output

# Buat direktori output jika belum ada
os.makedirs(output_dir, exist_ok=True)

# Dictionary untuk menyimpan path gambar dan label per kelas
class_images = defaultdict(list)
class_labels = defaultdict(list)

# Memuat semua gambar dan label ke dalam dictionary berdasarkan kelas
for filename in os.listdir(data_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(data_dir, filename)
        label_path = os.path.join(data_dir, os.path.splitext(filename)[0] + '.txt')
        
        # Ambil class_id dari nama file, misalnya '00266.jpg'
        class_id = int(filename.split('.')[0])  # Ambil class_id dari nama file
        
        class_images[class_id].append(image_path)
        class_labels[class_id].append(label_path)

# List untuk menyimpan path gambar yang akan dipindahkan ke train dan test
train_images = []
train_labels = []
test_images = []
test_labels = []

# Memilih sampel untuk train dan test
for class_id, images in class_images.items():
    total_images = len(images)
    
    # Acak urutan gambar
    random.shuffle(images)
    
    # Ambil 20% untuk test
    num_test = int(0.2 * total_images)
    test_selected = images[:num_test]
    
    # Sisanya untuk train
    train_selected = images[num_test:]
    
    train_images.extend(train_selected)
    train_labels.extend([label for img in train_selected for label in class_labels[class_id]])
    test_images.extend(test_selected)
    test_labels.extend([label for img in test_selected for label in class_labels[class_id]])

# Fungsi untuk memindahkan gambar dan label ke direktori output
def move_files(images, labels, output_folder):
    for img_path, lbl_path in zip(images, labels):
        img_filename = os.path.basename(img_path)
        lbl_filename = os.path.basename(lbl_path)
        
        output_img_path = os.path.join(output_folder, img_filename)
        output_lbl_path = os.path.join(output_folder, os.path.splitext(img_filename)[0] + '.txt')
        
        shutil.copyfile(img_path, output_img_path)
        shutil.copyfile(lbl_path, output_lbl_path)

# Direktori train dan test di dalam output_dir
train_dir = os.path.join(output_dir, 'train')
test_dir = os.path.join(output_dir, 'test')

# Buat direktori train dan test jika belum ada
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Pindahkan gambar dan label ke train
move_files(train_images, train_labels, train_dir)

# Pindahkan gambar dan label ke test
move_files(test_images, test_labels, test_dir)

# Tampilkan jumlah gambar per kelas di train dan test
train_class_counts = defaultdict(int)
test_class_counts = defaultdict(int)

for lbl_path in train_labels:
    with open(lbl_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            class_id = int(line.strip().split()[0])
            train_class_counts[class_id] += 1

for lbl_path in test_labels:
    with open(lbl_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            class_id = int(line.strip().split()[0])
            test_class_counts[class_id] += 1

print("Train dataset:")
print(f"Total images: {len(train_images)}")
print("Class distribution:")
print(train_class_counts)

print("\nTest dataset:")
print(f"Total images: {len(test_images)}")
print("Class distribution:")
print(test_class_counts)


Train dataset:
Total images: 741
Class distribution:
defaultdict(<class 'int'>, {7: 41, 18: 39, 34: 12, 33: 16, 38: 87, 13: 83, 14: 32, 2: 80, 12: 85, 19: 2, 1: 79, 21: 5, 11: 38, 3: 30, 9: 42, 26: 18, 41: 7, 28: 12, 15: 15, 0: 4, 10: 80, 8: 56, 4: 68, 23: 21, 35: 20, 25: 31, 40: 10, 5: 54, 36: 10, 42: 11, 30: 16, 39: 6, 17: 29, 32: 8, 16: 8, 24: 5, 20: 9, 29: 5, 22: 12, 6: 19, 27: 3, 31: 2, 37: 2})

Test dataset:
Total images: 0
Class distribution:
defaultdict(<class 'int'>, {})


In [11]:
import yaml

# Path ke direktori dataset output
dataset_dir = '/kaggle/working/dataset2'

# Definisikan konfigurasi YAML
config = {
    'path': dataset_dir,
    'train': f'{dataset_dir}/train',
    'val': f'{dataset_dir}/test',
    'nc': 43,  # Jumlah kelas yang Anda miliki
    'names': ['class0', 'class1', 'class2', 'class3', 'class4', 'class5', 
              'class6', 'class7', 'class8', 'class9', 'class10', 'class11', 
              'class12', 'class13', 'class14', 'class15', 'class16', 'class17', 
              'class18', 'class19', 'class20', 'class21', 'class22', 'class23', 
              'class24', 'class25', 'class26', 'class27', 'class28', 'class29', 
              'class30', 'class31', 'class32', 'class33', 'class34', 'class35', 
              'class36', 'class37', 'class38', 'class39', 'class40', 'class41', 
              'class42']  # Nama kelas sesuai dengan urutan indeks
}

# Tulis konfigurasi ke dalam file YAML
yaml_file = '/kaggle/working/dataset2/dataset.yaml'
with open(yaml_file, 'w') as file:
    yaml.dump(config, file)

print("YAML file created successfully.")


YAML file created successfully.


In [12]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.33 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5689.3/8062.4 GB disk)


In [13]:
import os
from ultralytics import YOLO

data_path = '/kaggle/working/dataset2/dataset.yaml'
model_path = 'yolov8s.pt'

model = YOLO(model_path)

model.train(
    data=data_path,
    epochs=5000,
    imgsz=640,
    batch=64,
    fliplr=0.0,    # Hindari flipping horizontal (mirroring)
    flipud=0.0,    # Hindari flipping vertikal (mirroring)
    degrees=0.0,   # Hindari rotasi
)


100%|██████████| 21.5M/21.5M [00:00<00:00, 173MB/s]


Ultralytics YOLOv8.2.33 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/dataset2/dataset.yaml, epochs=5000, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

100%|██████████| 755k/755k [00:00<00:00, 14.3MB/s]
2024-06-16 20:46:31,573	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-16 20:46:32,537	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=43

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_25/2784594751.py", line 9, in <module>
    model.train(
  File "/opt/conda/lib/python3.10/site-packages/ultralytics/engine/model.py", line 663, in train
  File "/opt/conda/lib/python3.10/site-packages/ultralytics/engine/model.py", line 184, in _get_hub_session
    """Transfers parameters with matching names and shapes from 'weights' to model."""
  File "/opt/conda/lib/python3.10/site-packages/ultralytics/hub/session.py", line 60, in __init__
    raise HUBModelError(f"model='{url}' not found. Check format is correct, i.e. "
ultralytics.utils.errors.HUBModelError: model='yolov8s.pt' not found. Check format is correct, i.e. model='https://hub.ultralytics.com/models/MODEL_ID' and try again.

During handling of the above exception, another exception occurred:

Traceback (most 